<a href="https://colab.research.google.com/github/nepslor/teaching/blob/main/TimeSeriesForecasting/advanced_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced models vs benchmarks for probabilistic forecasts
In this exercise we will train some advanced models on the M4 hourly dataset. The M4 hourly dataset has the following features:
* univariate time series
* scarce data (700 obs per series)
* different scales

We will
1. Create some benchmark forecasts
2. Fit two neural forecasters

Since we are in a data-scarce regime, both the benchmark model and the neural-based ones will be global models, that is, we will fit the same model on all the time series at once.



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random
import warnings
warnings.filterwarnings("ignore")
from itertools import product

In [ ]:
%%capture
!wget https://auto-arima-results.s3.amazonaws.com/M4-Hourly.csv
!wget https://auto-arima-results.s3.amazonaws.com/M4-Hourly-test.csv

In [ ]:
df_tr = pd.read_csv('M4-Hourly.csv')
df_te = pd.read_csv('M4-Hourly-test.csv').rename(columns={'y': 'y_test'})

n_series = 8
uids = df_tr['unique_id'].unique()[:n_series]
df_tr = df_tr.query('unique_id in @uids')
df_te = df_te.query('unique_id in @uids')
df_hat = df_te.copy()
df_hat['y_test'] = 0
df_tr.head()


,unique_id,ds,y
0,H1,1,605.0
1,H1,2,586.0
2,H1,3,586.0
3,H1,4,559.0
4,H1,5,511.0


## Benchmark forecaster
In the following we define a simple probabilistic benchmark model. The benchmark is the following:
* normalize the time series individually
* apply a moving average to detrend the series
* fit a linear model with a 2-days embedding
The probabilistic forecasts are obtained retrieving the bootstrapped errors on the training set

For each of the 8 selected series, we retrieve the NMAE:
$$nmae = \frac{\sum_{t=T}^{T+n_{sa}}\vert y_t-\hat{y}_t\vert}{\sum_{t=T}^{T+n_{sa}}\vert y_t \vert}$$

In [ ]:

def get_hankel(df, embedding=3):
  return pd.concat([df.shift(-l) for l in range(embedding)], axis=1).iloc[:-embedding]

lagged_mav = lambda x, k: x.copy().rolling(k).mean()


class LinearForecaster:
  """
  A simple linear forecaster: retrieves the signal's Hankel matrix of
  the requested embedding and fits a linear regression on future values.
  """
  def __init__(self, steps_ahead=1, embedding=3):
    self.steps_ahead = steps_ahead
    self.embedding = embedding
    self.theta = None
    self.mean = None
    self.std = None

  def fit(self, y0):
    y = y0.copy()
    self.mean = np.array(y.mean())
    self.std = np.array(y.std())
    y_embedded = get_hankel(y, self.steps_ahead + self.embedding)
    x = y_embedded.iloc[:, :self.embedding].values/self.std-self.mean
    target = y_embedded.iloc[:, self.embedding:].values
    self.theta = np.linalg.pinv(x.T@x)@(x.T@target)
    return self

  def predict(self, df):
    x = df.iloc[-self.embedding:].values.reshape(1, -1)/self.std-self.mean
    preds = x @ self.theta
    return pd.DataFrame(preds.ravel(), index=df.iloc[-self.steps_ahead:].index + np.arange(self.steps_ahead), columns=['y_hat_lin'])


class LinearDetrendedForecaster:
  def __init__(self, steps_ahead=13, linear_embedding=3):
    self.steps_ahead = steps_ahead
    self.linear_embedding = linear_embedding
    self.rt_model = LinearForecaster(steps_ahead=steps_ahead, embedding=linear_embedding)

  def fit(self, df):
    # STL decomposition of the signal
    trend = lagged_mav(df, self.steps_ahead)
    filt = ~trend.isna()
    self.rt_model.fit((df-trend).loc[filt])
    return self

  def predict(self, df):
    trend = lagged_mav(df, self.steps_ahead)
    y_hat = self.rt_model.predict(df-trend) + trend.iloc[-1]
    y_hat.name = 'lin_detrended'
    return y_hat


In [ ]:

# preprocessing
means, stds = {}, {}
df_tr_norm = df_tr.copy()
for uid in df_tr['unique_id'].unique():
    means[uid] = df_tr.query('unique_id == @uid')['y'].mean()
    stds[uid] = df_tr.query('unique_id == @uid')['y'].std()
    df_tr_norm.loc[df_tr['unique_id']==uid, 'y'] = (df_tr.query('unique_id == @uid')['y'] - means[uid])/stds[uid]


lm = LinearDetrendedForecaster(steps_ahead=48, linear_embedding=48).fit(df_tr_norm['y'])


In [ ]:
# predict point forecasts
for uid in df_tr['unique_id'].unique():
    y_hat = lm.predict(df_tr_norm.loc[df_tr_norm['unique_id'] == uid, 'y']) * stds[uid] + means[uid]
    df_hat.loc[df_hat['unique_id'] == uid, 'y_test'] = y_hat.values

# retrieve errors on the training set
err_pdf = {}
q_vect = [0.05, 0.1, 0.9, 0.95]
for uid in df_tr['unique_id'].unique():
  errs_u = []
  for i in range(700-48*3):
    y_past = df_tr_norm.loc[df_tr_norm['unique_id'] == uid, 'y'].iloc[:48*2+i]
    y_te = df_tr_norm.loc[df_tr['unique_id'] == uid, 'y'].iloc[48*2+i:48*2+i + 48]
    y_hat = lm.predict(y_past)
    errs_u.append((y_te.values-y_hat.values.ravel()) * stds[uid])
  err_pdf[uid] = np.quantile(np.vstack(errs_u).T,q_vect, axis=1).T

In [ ]:
nmae = lambda x,y: np.mean(np.abs(x-y))/(np.mean(x)+1e-6)
nmaes_lin = {}
# plot results
fig, ax = plt.subplots(4, 2, figsize=(18, 12))
plt.subplots_adjust(hspace=0.25)
for uid, a in zip(df_tr['unique_id'].unique(), ax.ravel()):
    y_te = df_te.loc[df_hat['unique_id'] == uid, 'y_test']
    y_hat = df_hat.loc[df_hat['unique_id'] == uid, 'y_test']
    y_te.plot(ax=a)
    a.plot(y_hat.index, err_pdf[uid] + y_hat.values.reshape(-1, 1), color='red', alpha=0.2)
    y_hat.plot(ax=a)
    nmaes_lin[uid] = nmae(y_te.values.ravel(), y_hat.values.ravel())
    a.set_title('{} NMAE:{:0.2e}'.format(uid, nmaes_lin[uid]))

## ❓ARIMA benchmark models
Create a benchmark forecaster using the [darts](https://unit8co.github.io/darts/#example-usage) library or the [statsmodels](https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMA.html) implementation.
Train the model on `df_tr` and test and plot the forecasts on `df_te` along with the NMAE score.

# Probabilistic forecasts with neural models and quantile loss
In the following we will train recursive neural models using the `NeuralForecast` library, using the quantile loss function:

$$\rho(y,\hat{q},\tau) = (\tau-1) \sum_{y_i<\hat{q}
}\left(y_i-\hat{q}\right)+\tau \sum_{y_i \geq \hat{q}}\left(y_i-\hat{q}\right)$$


We will fit:
* a naive Long Short Term Memory Network [(LSTM)](https://arxiv.org/abs/2201.12886)
* the Neural Hierarchical Interpolation [(NHITS)](https://arxiv.org/abs/2201.12886) model


In [ ]:
!pip install git+https://github.com/Nixtla/neuralforecast.git@main

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MQLoss
from neuralforecast.models import LSTM, DilatedRNN, NHITS

In [ ]:
horizon = 48
levels = [80, 90]
models = [LSTM(input_size=-1, h=horizon,
               loss=MQLoss(level=levels), max_steps=300),
          NHITS(input_size=7*horizon, h=horizon,
                n_freq_downsample=[24, 12, 1],
                loss=MQLoss(level=levels), max_steps=200),]
fcst = NeuralForecast(models=models, freq=1)

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [ ]:
fcst.fit(df=df_tr)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
forecasts = fcst.predict()
forecasts = forecasts.reset_index()
forecasts.shape

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

(384, 12)

In [ ]:

def plot_results(model):
  nmaes = {}
  q_names = ['lo-90','lo-80','hi-90','hi-80']
  fig, ax = plt.subplots(4, 2, figsize=(18, 12))
  plt.subplots_adjust(hspace=0.25)
  for uid, a in zip(df_tr['unique_id'].unique(), ax.ravel()):
      y_te = df_te.loc[df_te['unique_id'] == uid, 'y_test']
      y_hat = forecasts.loc[forecasts['unique_id'] == uid, '{}-median'.format(model)]
      y_te.plot(ax=a)
      [a.plot(forecasts.loc[forecasts['unique_id'] == uid, '{}-{}'.format(model, q_str)], color='red', alpha=0.2) for q_str in q_names]
      y_hat.plot(ax=a)
      nmaes[uid] = nmae(y_te.values.ravel(), y_hat.values.ravel())
      a.set_title('{} NMAE:{:0.2e}'.format(uid,nmaes[uid]))
  plt.suptitle(model)
  return nmaes

nmaes_lstm = plot_results('LSTM')
nmaes_nhits = plot_results('NHITS')

In [ ]:
pd.concat([pd.DataFrame(nmaes_lin, index=['lin']), pd.DataFrame(nmaes_lstm, index=['lstm']), pd.DataFrame(nmaes_nhits, index=['nhits'])]).T.plot()